In [1]:
import pandas as pd
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from sklearn.cluster import KMeans
import helpers
import pickle
import pathlib

Using Russia server backend.


In [2]:
wish_list_df = pd.read_csv('data/db/wish_lists.csv')
wish_list_df = wish_list_df[['id', 'title']]
print('wish lists:', len(wish_list_df))

wish_list_df.head()

wish lists: 2405


,id,title
0,d461b35a519d8af010947904,For the soul
1,cd5fae468ca89c6478416068,Для дома
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей
3,945fdb57abc3e52529717750,Шмотки
4,505fc79cdff2233891049727,📱Техника


In [3]:
w_titles = wish_list_df['title'].to_list()
wish_list_df['new_title'] = [helpers.clear_text(title) for title in w_titles]
wish_list_df.head()

,id,title,new_title
0,d461b35a519d8af010947904,For the soul,for the soul
1,cd5fae468ca89c6478416068,Для дома,для дома
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей,для детей
3,945fdb57abc3e52529717750,Шмотки,шмотки
4,505fc79cdff2233891049727,📱Техника,техника


In [4]:
def get_lang_detector(nlp, name):
    return LanguageDetector()


lang_nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
lang_nlp.add_pipe('language_detector', last=True)

In [5]:
langs = []
for title in w_titles:
    doc = lang_nlp(title)
    langs.append(doc._.language['language'])

wish_list_df['lang'] = langs
wish_list_df.head()

,id,title,new_title,lang
0,d461b35a519d8af010947904,For the soul,for the soul,en
1,cd5fae468ca89c6478416068,Для дома,для дома,uk
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей,для детей,ru
3,945fdb57abc3e52529717750,Шмотки,шмотки,mk
4,505fc79cdff2233891049727,📱Техника,техника,ru


In [6]:
print('all wishes:', len(wish_list_df))
wish_list_df = wish_list_df[wish_list_df['new_title'] != '']
print('filtered:', len(wish_list_df))

all wishes: 2405
filtered: 2347


In [7]:
need_translation = True

translations_file = 'data/result/translations.pkl'
my_file = pathlib.Path(translations_file)
if my_file.is_file():
    with open(translations_file, 'rb') as f:
        translations = pickle.load(f)
    need_translation = False

need_translation = True
if need_translation:
    to_translate = wish_list_df[~wish_list_df['lang'].isin(['ru', 'UNKNOWN'])]['new_title'].unique()
    print('Titles to translate:', len(to_translate))
    print(to_translate[:3])

    translations = {}
    for i, title in enumerate(to_translate):
        russian_text = helpers.ru_text(title)
        translations[title] = russian_text
        if i % 50 == 0:
            print(f'{i} wishlist translated')

    with open(translations_file, 'wb') as f:
        pickle.dump(translations, f)
    print(f'translations are saved to {translations_file}')

else:
    print(f'translations are loaded from {translations_file}')

Titles to translate: 646
['for the soul' 'для дома' 'шмотки']
0 wishlist translated
50 wishlist translated
100 wishlist translated
150 wishlist translated
200 wishlist translated
250 wishlist translated
300 wishlist translated
350 wishlist translated
400 wishlist translated
450 wishlist translated
500 wishlist translated
550 wishlist translated
600 wishlist translated
translations are saved to data/result/translations.pkl


In [8]:
def get_ru_text(row):
    if row['lang'] in ['ru', 'UNKNOWN']:
        return row['new_title']
    else:
        new_title = translations[row['new_title']]
        return new_title.strip().lower()

wish_list_df['ru_title'] = wish_list_df.apply(lambda row: get_ru_text(row), axis=1)
wish_list_df.head()

,id,title,new_title,lang,ru_title
0,d461b35a519d8af010947904,For the soul,for the soul,en,для души
1,cd5fae468ca89c6478416068,Для дома,для дома,uk,для дома
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей,для детей,ru,для детей
3,945fdb57abc3e52529717750,Шмотки,шмотки,mk,шмотки
4,505fc79cdff2233891049727,📱Техника,техника,ru,техника


In [9]:
nlp = spacy.load('ru_core_news_md')

stop_words = {'подарки', 'подарок', 'подарить', 'подарочки', 'список',
              'всякий', 'другим', 'других', 'желание', 'пожелание', 'надо',
              'очень', 'хочу', 'хотеть', 'хотеться', 'нг', 'др',
              'день рождение', 'любой', 'идея', 'виш', 'прочие', 'прочая',
              'вишлист', 'виш лист', 'разное', 'хороший', 'желания', 'идея',
              'новогодний', 'тест', 'test', 'zz', 'zzz', 'yyy', 'ы', 'a',
              'купить', 'покупка', 'продать', 'покупать', 'вишлистик',
              'получить', 'всегда', 'tmp'}
nlp.Defaults.stop_words |= stop_words

ru_names = pd.read_json('data/names_table.jsonl', lines=True)
names = sorted(ru_names['text'])
names = [name.lower() for name in names]

print('ru model is loaded')

ru model is loaded


In [10]:
wish_list_df['new'] = wish_list_df.apply(lambda row: helpers.wishlist_text(nlp, row['ru_title'], names), axis=1)
wish_list_df.head()

,id,title,new_title,lang,ru_title,new
0,d461b35a519d8af010947904,For the soul,for the soul,en,для души,душа
1,cd5fae468ca89c6478416068,Для дома,для дома,uk,для дома,дом
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей,для детей,ru,для детей,ребёнок
3,945fdb57abc3e52529717750,Шмотки,шмотки,mk,шмотки,шмотки
4,505fc79cdff2233891049727,📱Техника,техника,ru,техника,техника


In [12]:
wish_list_df = wish_list_df[['id', 'title', 'lang', 'new']]
file_to_save = 'data/result/new_wishlists.csv'
wish_list_df.to_csv(file_to_save)
print(f'wishlists are saved to {file_to_save}')

wish_list_df.head()

wishlists are saved to data/result/new_wishlists.csv


,id,title,lang,new
0,d461b35a519d8af010947904,For the soul,en,душа
1,cd5fae468ca89c6478416068,Для дома,uk,дом
2,5c5fc79ce81a1a1030838078,👶🏻 Для детей,ru,ребёнок
3,945fdb57abc3e52529717750,Шмотки,mk,шмотки
4,505fc79cdff2233891049727,📱Техника,ru,техника


In [13]:
titles = wish_list_df[wish_list_df['new'] != '']['new'].unique()
wish_list_vecs = [nlp(wish_list).vector for wish_list in titles]
print('vectors are ready')

vectors are ready


In [14]:
cluster_wishlists = pd.DataFrame({'title': titles})
cluster_count = 100
kmeans = KMeans(n_clusters=cluster_count, random_state=0).fit(wish_list_vecs)
cluster_wishlists['label'] = kmeans.labels_

cluster_wishlists.head()

,title,label
0,душа,1
1,дом,1
2,ребёнок,42
3,шмотки,4
4,техника,87


In [15]:
for i in range(cluster_count):
    cluster_wish_lists = cluster_wishlists[cluster_wishlists['label'] == i]['title'].to_list()
    label_count = len(cluster_wish_lists)
    print(f'Label {i}: {label_count} wish lists')
    for wishlist in cluster_wish_lists:
        print(wishlist)
    print('\n\n')

Label 0: 2 wish lists
эко
ик



Label 1: 40 wish lists
душа
дом
новый
ходить
делать
другое
большой
цель
любить любить
дело
момент
место
муж
первый
комната
семья
друг
новый дом
почти тридцать
домой
случай
семья дом
мечтать
двигаться
скорее цель
а
знать
час
будущее
иметь
про
просто
место где побывать
можно
прочее
видеть
семья друг
первый дело
лицо
новый жизнь



Label 2: 204 wish lists
хирургичка
3000 ₽
деревообрабатывание
< 1500
тестовый задание
настолки
нехудожественный литература
палитта
плакать техно
медсестра
хотелки
устройство
pandora
чашка
животное
g.a.s
андреши
$ $ $
хочушки
шуга дэддь
сладкий 24
хуйня собачий
кататься конёк
дом досуг
< 8000
мнемнемне
масоки
электроника
космос
косплей
посетить
вовке
любы
34-й
обустройство дом
хватать
аниме
12.12
нашпщрпшпон изд группа лит рот
даааам
general хотелки
ювелирка
алёне
марка
дэрэ
пашуле годик
главный
нога
т.п
кросовки
уют
фандомный мерч
запрещенка
папе
10 000 ₽
кируша
aliexpress
10к
гггс
фотосъёмка
аромат
10 +
wishlist косметос
кицы
бо